# Hourly Data #

In [ ]:
# imports
import pandas as pd
import numpy as np
import time
from timeit import default_timer as timer
import gc

In [ ]:
# define data location
#location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [ ]:
# hourly data
use_jan_in = pd.read_excel(location+'Zip_HourlylUsage_2018.01.xlsx')
use_jul_in = pd.read_excel(location+'Zip_HourlylUsage_2018.07.xlsx')

# daily data
use_oct_in = pd.read_excel(location+'Sample Usage_2017.10 Oct.xlsx')
use_nov_in = pd.read_excel(location+'Sample Usage_2017.11 Nov.xlsx')
use_dec_in = pd.read_excel(location+'Sample Usage_2017.12 Dec.xlsx')
# use_jan_in = pd.read_excel(location+'Sample Usage_2018.01 Jan.xlsx')
use_feb_in = pd.read_excel(location+'Sample Usage_2018.02 Feb.xlsx')
use_mar_in = pd.read_excel(location+'Sample Usage_2018.03 March.xlsx')
use_apr_in = pd.read_excel(location+'Sample Usage_2018.04 April.xlsx')
use_may_in = pd.read_excel(location+'Sample Usage_2018.05 May.xlsx')
use_jun_in = pd.read_excel(location+'Sample Usage_2018.06 June.xlsx')
# use_jul_in = pd.read_excel(location+'Sample Usage_2018.07 Jul.xlsx')
use_aug_in = pd.read_excel(location+'Sample Usage_2018.08 Aug.xlsx')
use_sep_in = pd.read_excel(location+'Sample Usage_2018.09 Sep.xlsx')

In [ ]:
# merge use data
daily_in = [use_oct_in, use_nov_in, use_dec_in, use_feb_in, use_mar_in, 
            use_apr_in, use_may_in, use_jun_in, use_aug_in, use_sep_in]
daily = pd.concat(daily_in)
daily = daily.rename(columns={'DAccountID':'DACCOUNTID', 'DMeterNo':'DMETERNO',
                              'DAILY_INTERVAL_USAGE':'Use'})

hourly_in = [use_jan_in, use_jul_in]
hourly = pd.concat(hourly_in)

In [ ]:
del (use_oct_in, use_nov_in, use_dec_in, use_feb_in, use_mar_in, 
     use_apr_in, use_may_in, use_jun_in, use_aug_in, use_sep_in, 
     use_jan_in, use_jul_in, daily_in, hourly_in)

In [ ]:
# create ID tuple
ids = pd.Series(list(map(tuple, daily[['DACCOUNTID', 'DMETERNO']].values)))
daily['ID'] = ids.values

ids = pd.Series(list(map(tuple, hourly[['DACCOUNTID', 'DMETERNO']].values)))
hourly['ID'] = ids.values

In [ ]:
# find gas records
daily = daily.loc[daily['UOM'] == 'CCF']
daily = daily.drop(columns=['UOM'])
hourly = hourly.loc[hourly['UOM'] == 'CCF']
hourly = hourly.drop(columns=['UOM'])

# convert to datetime
daily['Dt'] =  pd.to_datetime(daily['METERREADDATE'])
hourly['Dt'] =  pd.to_datetime(hourly['METERREADDATE'])

In [ ]:
# load customer data
customer_in = pd.read_excel(location+'PECO Zip Customer 2018.10.01 v2.xlsx', sheet_name="Account")

# create id tuple
ids = pd.Series(list(map(tuple, customer_in[['DACCOUNTID', 'DMETERNO']].values)))
customer_in['ID'] = ids.values
customer = customer_in.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'COUNTYCODE'])
del customer_in

## Data Restructuring ##

In [ ]:
# functions for naming consistency
def decrement(x, startswith, split):
    """
    decrements a passed string of form "demo#" by 1
    
    Parameters
    ----------
    x : string to be decremented
    split : string to split on

    Returns
    ----------
    y : decremented string
    """
    if x.startswith(startswith):
        a,b = x.split(split)
        b = int(b)-1
        y = a + split + str(b)

        return y

    else:
        return x

    
def interval_to_hour(df):
    """
    function for fast rename/relabel during tidying process
    
    Parameters
    ----------
    df : pandas data frame

    Returns
    ----------
    df : data frame with updated column names
    """
    
    df = df.rename(columns=lambda x: decrement(x, "INTERVAL_", "_"))
    df = df.rename(columns=lambda x: x.replace("INTERVAL_", "HR"))
    return df

In [ ]:
# rename for consistency
hourly = interval_to_hour(hourly)
hourly = hourly.drop(columns=['METERREADDATE','HR24'])
daily = daily.drop(columns=['METERREADDATE'])

In [ ]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_hourly = pd.melt(hourly, 
                      id_vars=['ID','DACCOUNTID','DMETERNO','Dt'],
                      var_name='Hour', value_name='Use')

tidy_hourly_agg = tidy_hourly.groupby(['ID','DACCOUNTID','DMETERNO','Dt']).sum().reset_index()
#tidy_hourly_agg.head()

In [ ]:
# join hourly & daily
dfs = [tidy_hourly_agg, daily]
use = pd.concat(dfs, keys=['ID','DACCOUNTID', 'DMETERNO', 'Dt', 'Use'], sort=True).reset_index(drop=True)
#use.head()

In [ ]:
# # merge use data
# # hourly_in = [use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
# #              use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in]
# use = pd.concat([use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
#                  use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in])

In [ ]:
who_ls

In [ ]:
# # drop unused tables for memory space
# del (use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
#      use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in)

In [ ]:
# # save as pickle so we don't have to do this again
# use.to_pickle(location+'hourly_use_raw.pkl.zip')
# del use

In [ ]:
# load pickle
start = timer()

use = pd.read_pickle(location+'daily_use_raw.pkl.zip')

end = timer()
print((end - start)/60) # Time in minutes (.19 - MUCH FASTER!!)

In [ ]:
# append rate code data
use = use.merge(customer, how='inner', on=['ID','DACCOUNTID','DMETERNO'])
#use.head()

In [ ]:
# Add dummy variables for day-of-week
#use = use.join(pd.get_dummies(use['Dt'].dt.weekday_name))
use['Weekday'] = pd.get_dummies(use['Dt'].dt.weekday < 5)[True]

# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
dr = pd.date_range(start=min(use['Dt']), end=max(use['Dt']))
holidays = cal.holidays(start=dr.min(), end=dr.max())

use['Holiday'] = pd.get_dummies(use['Dt'].isin(holidays))[True]
#use = use.rename(columns={True:'Holiday'})
#use = use.drop(columns=[False])
#use.head()

In [ ]:
# sort by date & time
use = use.sort_values(by=["Dt"])

#### Quick look at amount of data we have per ID ####

In [ ]:
# drop records with missing data
if len(use) == len(use.dropna(subset=['Use'])):
    print(f'There is no missing data in the {len(use)} rows of useage data')
else:
    use = use.dropna(subset=['Use'])

In [ ]:
print(f"There are {len(use['DACCOUNTID'].drop_duplicates())} unique AccountIDs in the data")
print(f"There are {len(use['DMETERNO'].drop_duplicates())} unique MeterNos in the data")
ids = use['ID'].drop_duplicates()
print(f"There are {len(ids)} unique AccountID / MeterNo pairs in the data")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ids_to_plot = use.groupby('ID').count()
sns.distplot(ids_to_plot['Use'], bins=20, kde=False, rug=False)
plt.title('Frequency of IDs with X dates')

In [ ]:
sns.distplot(ids_to_plot[ids_to_plot['Use'] > 330]['Use'], bins=20, kde=False, rug=False)
plt.title('Frequency of IDs with at least 330 dates')

In [ ]:
#for i in range(1,365):
#    print(f"{ids_to_plot[ids_to_plot['Use'] >= i]['Use'].count()} records have >={i} dates")

#### Find IDs with sufficient data to build initial segments on --> reliable training set ####

In [ ]:
# find IDs with at least 360 days of data
sufficient_ids = ids_to_plot[ids_to_plot['Use'] >= 360].reset_index()['ID']
print(f"{100*len(sufficient_ids)/len(use['ID'].unique())} percent of IDs have at least 360 days of data")

In [ ]:
# find the dates for which sufficient_ids all have data
pivoted = (use[use['ID'].isin(sufficient_ids)]
           .pivot_table(index='Dt', columns=['ID'], values='Use', fill_value=None))
sufficient_dates = pivoted.dropna().reset_index()['Dt']
print(f"The IDs with sufficient data overlap on {len(sufficient_dates)} days of the most recent year of data")

In [ ]:
# filter total use dataset to contain only IDs who have sufficient data and only dates where sufficient_ids have data
sufficient_use = use[(use['ID'].isin(sufficient_ids)) & (use['Dt'].isin(sufficient_dates))]
#sufficient_use.head()

In [ ]:
#sanity check
print(len(use))
print(len(sufficient_use))
print(len(ids))
print(len(sufficient_ids))

### Subgroup detection ###

In [ ]:
print(use[['ID','REVENUCODE']].drop_duplicates().groupby('REVENUCODE').count())
print("nan           "+str(len(use[np.isnan(use['REVENUCODE'])][['ID']].drop_duplicates())))

In [ ]:
print(use[['ID','REVENUCODE']].drop_duplicates().groupby('REVENUCODE').count())
print("nan           "+str(len(use[np.isnan(use['REVENUCODE'])][['ID']].drop_duplicates())))

In [ ]:
# recode 'REVENUCODE' as 'TYPE'
def recode(df):
    """combines REVENUCODE groups 1,12,nan to HOME and 3,5 to COMMERICAL"""
    df['TYPE'] = None
    df.loc[ df.REVENUCODE == 1, 'TYPE' ] = 'HOME'
    df.loc[ df.REVENUCODE == 12, 'TYPE' ] = 'HOME'
    df.loc[ np.isnan(df['REVENUCODE']), 'TYPE' ] = 'HOME'
    df.loc[ df.REVENUCODE == 3, 'TYPE' ] = 'COMM'
    df.loc[ df.REVENUCODE == 5, 'TYPE' ] = 'COMM'
    
    #df['TYPE'][df['REVENUCODE']==1] = 'HOME'
    #df['TYPE'][df['REVENUCODE']==12] = 'HOME'
    #df['TYPE'][np.isnan(df['REVENUCODE'])] = 'HOME'
    #df['TYPE'][df['REVENUCODE']==3] = 'COMM'
    #df['TYPE'][df['REVENUCODE']==5] = 'COMM'
    
    return df

In [ ]:
# ignore errors
use = recode(use)
sufficient_use = recode(sufficient_use)

### Save ###

In [ ]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
use.to_pickle(location+'peco_daily.pkl.zip')
sufficient_use.to_pickle(location+'peco_sufficient_daily.pkl.zip')
ids.to_pickle(location+'peco_daily_ids.pkl.zip')
sufficient_ids.to_pickle(location+'peco_sufficient_daily_ids.pkl.zip')

 ### Merge use & weather? ###

In [ ]:
# load data
daily = pd.read_pickle(location+'peco_daily.pkl.zip')
sufficient_daily = pd.read_pickle(location+'peco_sufficient_daily.pkl.zip')
weather = pd.read_pickle(location+'daily_weather.pkl.zip')

In [ ]:
# merge
daily = pd.merge(daily, weather, how='inner', on=['Dt'])
sufficient_daily = pd.merge(sufficient_daily, weather, how='inner', on=['Dt'])
# CustIDs || Date | Consumption |||| Weather_variables